<h1>Trabalho Prático 1 de Algoritmos 2<h1>


Matrícula: 2019041647 <br>
Nome: Roberth Alves Parreiras


<h2>Envoltória Convexa</h2>

In [ ]:
import numpy as np


#encontra o primeiro ponto que vai estar na envoltória
def PontoMaisAEsquerda(pontos):

    min = 0
    for i in range(len(pontos)):
        if pontos[i,0] < pontos[min,0]:
            min = i
        elif pontos[i,0] == pontos[min,0]:

            #testa qual o menor y
            if pontos[i,1] < pontos[min,1]:
                min = i

    return min



#retorna a direcao do caminho entre os pontos dados
def Orientacao(p0, p1, p2):

    #cria numpy array para facilitar o cálculo
    pt0 = np.empty((0,2), float)
    pt1 = np.empty((0,2), float)
    pt2 = np.empty((0,2), float)

    pt0 = np.vstack((pt0, p0))
    pt1 = np.vstack((pt1, p1))
    pt2 = np.vstack((pt2, p2))

    val = ((pt1[0,0] - pt0[0,0]) * (pt2[0,1] - pt0[0,1]) - (pt2[0,0] - pt0[0,0]) * (pt1[0,1] - pt0[0,1]))

    if val > 0:
        return 'horario'
    elif val < 0:
        return 'antiHorario'
    else:
        return 'colinear'    
 

#retorna as posições dos pontos que pertencem a envoltória
def EnvoltoriaConvexa(pontos):

    ptEsquerda = PontoMaisAEsquerda(pontos)
    envoltoria = [] #array para guardar as posições que estão na envoltória
     
    pt = ptEsquerda
    q = 0

    #testa e adiciona os pontos que vão estar na envoltória
    while(True):
         
        envoltoria.append(pt)
        q = (pt + 1) % len(pontos)

        for i in range(len(pontos)):

            if(Orientacao(pontos[pt], pontos[i], pontos[q]) == 'antiHorario'):
                q = i
 
        pt = q

        #retorna após checar todos os pontos
        if(pt == ptEsquerda):
            envoltoria.append(pt) #adiciona a primeira posição para fechar a envoltória
            return envoltoria
 


<h2>Teste de intersecção<h2>

Logo após encontrar a envoltória convexa de cada classe, é testado se essas envoltórias se intersectam, sendo retornado verdadeiro caso se intersectem.
<p>O algoritmo abaixo faz esse teste:

In [ ]:
#retorna o produto vetorial entre os pontos dados
def Direcao(pi, pj, pk):
    
    val1 = pk - pi
    val2 = pj - pi

    return np.cross(val1, val2)


#retorna se os segmentos são colineares ou não
def NoSegmento(pi, pj, pk):

    #cria numpy array para facilitar o cálculo
    ptI = np.empty((0,2), float)
    ptJ = np.empty((0,2), float)
    ptK = np.empty((0,2), float)

    ptI = np.vstack((ptI, pi))
    ptJ = np.vstack((ptJ, pj))
    ptK = np.vstack((ptK, pk))

    if min(ptI[0,0], ptJ[0,0]) <= ptK[0,0] <= max(ptI[0,0], ptJ[0,0]) and min(ptI[0,1], ptJ[0,1]) \
    <= ptK[0,1] <= max(ptI[0,1], ptJ[0,1]):
        return True
    else:
        return False



#retorna se os segmentos se intersectam
def IntercesaoDeSegmentos(p1, p2, p3, p4):
    d1 = Direcao(p3, p4, p1)
    d2 = Direcao(p3, p4, p2)
    d3 = Direcao(p1, p2, p3)
    d4 = Direcao(p1, p2, p4)

    if ((d1 > 0 and d2 < 0) or (d1 < 0 and d2 > 0)) and ((d3 > 0 and d4 < 0) or (d3 < 0 and d4 > 0)):
        return True
    elif d1 == 0 and NoSegmento(p3, p4, p1):
        return True
    elif d2 == 0 and NoSegmento(p3, p4, p2):
        return True
    elif d3 == 0 and NoSegmento(p1, p2, p3):
        return True
    elif d4 == 0 and NoSegmento(p1, p2, p4):
        return True
    else:
        return False


In [ ]:
def separabilidadeLinear(pontos, pontos2):

    intersectam = False

    #ordena os pontos
    pontosXOrdenados = pontos[pontos[:,0].argsort()]
    pontos2XOrdenados = pontos2[pontos2[:,0].argsort()]

    pontosYOrdenados = pontos[pontos[:,1].argsort()]
    pontos2YOrdenados = pontos2[pontos2[:,1].argsort()]
    
    #testa se uma envoltória está dentro da outra envoltória
    if ((pontosXOrdenados[0,0] <= pontos2XOrdenados[0,0] or pontos2XOrdenados[0,0] <= pontosXOrdenados[0,0]) and (pontosXOrdenados[-1,0] >=     pontos2XOrdenados[-1,0] or pontos2XOrdenados[-1,0] >= pontosXOrdenados[-1,0])) \
        and ((pontosYOrdenados[0,1] <= pontos2YOrdenados[0,1] or pontos2YOrdenados[0,1] <= pontosYOrdenados[0,1]) and (pontosYOrdenados[-1,1] >= pontos2YOrdenados[-1,1] or pontos2YOrdenados[-1,1] >= pontosYOrdenados[-1,1])):
        
        intersectam = True
        return intersectam
        
    #testa se os segmentos se intersectam
    for i in range(len(pontos) - 1):
        for j in range(len(pontos2) - 1):
            intersectam = IntercesaoDeSegmentos(pontos[i], pontos[i+1], pontos2[j], pontos2[j+1])

    return intersectam



<h2>Pontos mais próximos

Com as envoltórias não se intersectando, tem-se a procura dos pontos mais próximos entre as duas envoltórias.
<p>O algoritmo abaixo encontra esses pontos:

In [ ]:
import sys, math

#calcula a distância euclidiana entre os pontos
def distancia(p1, p2):

    pt1 = np.empty((0,2), float)
    pt2 = np.empty((0,2), float)

    pt1 = np.vstack((pt1, p1))
    pt2 = np.vstack((pt2, p2))

    return math.sqrt((pt1[0,0] - pt2[0,0]) * (pt1[0,0] - pt2[0,0]) + (pt1[0,1] - pt2[0,1]) * (pt1[0,1] - pt2[0,1]))


#calcula o ponto mais próximo entre as envoltórias
def proximo(pt1, pt2):

    min_val = sys.maxsize
    pontos_proximos = np.empty((0,2))


    for i in range(len(pt1)):
        for j in range(len(pt2)):
            if distancia(pt1[i], pt2[j]) < min_val:
                min_val = distancia(pt1[i], pt2[j])
                pontos_proximos = np.vstack((pt1[i], pt2[j]))
    
    return pontos_proximos
    

<h2>Equação da reta 

Com os pontos mais próximos encontrados, agora é encontrar a equação da reta entre esses pontos.
<p>O algoritmo abaixo faz isso:

In [ ]:
#achar a equação da reta do tipo Ax + By + c = 0

#calcula a inclinação da reta que passa entre os pontos mais próximos das envoltórias
def InclinacaoDaReta(ptsProximos):
    inclinacao = np.array([ptsProximos[0,1] - ptsProximos[1,1], ptsProximos[0,0] - ptsProximos[1,0]])
    return inclinacao

#calcula o deslocamento dessa reta
def DeslocamentoDaReta(ptsProximos, Inclinacao):
    b = (ptsProximos[0,1] - ptsProximos[0,0] * Inclinacao)
    return b



<h2>Ponto médio

O algoritmo abaixo encontra o ponto médio:

In [ ]:
#encontrar o ponto médio

#calcula o ponto médio entre os pontos mais próximos que passa pela reta
def pontoMedio(ptsProximos):
    ptMedio = np.empty((0,2), float)
    pts = np.array([(ptsProximos[0,0] + ptsProximos[1,0])/2, (ptsProximos[0,1] + ptsProximos[1,1])/2])
    ptMedio = np.vstack((ptMedio, pts))

    return ptMedio


<h2>Equação da reta que é perpendicular ao ponto médio

Com a equação da reta e com o ponto médio, agora deve ser encontrado a equação da reta perpendicular a esse ponto médio.
<p>O algoritmo abaixo encontra essa equação da reta:

In [ ]:
#encontra a reta perdendicular a reta que passa pelos pontos mais próximos
def InclinacaoDaRetaPerpendicular(inclinacao):

    a = (inclinacao[1] * inclinacao[0]) / inclinacao[0]
    b = (-inclinacao[1] * inclinacao[0]) / inclinacao[1]

    return a,b


#calcula o valor c da reta
def ValorC(ptMedio, valAB):
    c = -ptMedio[0,0] * valAB[0,0] + ptMedio[0,1] * valAB[0,1]
    return c


In [ ]:
#classifica os pontos dados
def classificador(valAB, pontosDesconhecidos, pontosClassificados):
    
    if(valAB[0,1]):
        for i, valor in enumerate(pontosDesconhecidos):
            if(valor > 0):
                pontosClassificados[i] = 0
            elif(valor < 0):
                pontosClassificados[i] = 1
            else:
                pontosClassificados[i] = -1
    else:
        for i, valor in enumerate(pontosDesconhecidos):
            if(valor > 0):
                print(i)
                pontosClassificados[i] = 1
            elif(valor < 0):
                pontosClassificados[i] = 0
            else:
                pontosClassificados[i] = -1
    
    return pontosClassificados


<h1>Relatório

Esse trabalho prático tem como objetivo resolver o problema de encontrar uma envoltória convexa entre duas classes, e depois classificar a qual dessas duas classes esses pontos desconhecidos pertencem. Dessa forma, primeiramente, foi implementado a envoltória convexa de Jarvis, vista em aula, com sua complexidade de tempo *O*(P*E), sendo P a quantidade de pontos dado no input da função e E a quantidade de pontos retornado, sendo esses pontos retornados, os pontos da envoltória.
<p>Encontrando as envoltórias, testa se elas se intersectam, com o algoritmo de varredura, visto em aula, que tem complexidade de tempo <i>O</i>(E*E'), com E o número de pontos da envoltória de uma das classes e E' o número de pontos da envoltória de outra classe.
<p>Não se intersectando, acha-se o ponto de cada envoltória mais próximos, utilizando a distância de Euclides, para todos os pontos das envoltórias. Assim, a complexidade de tempo é <i>O</i>(E*E'), com E o número de pontos da envoltória de uma das classes e E' o número de pontos da envoltória de outra classe.
<p>Com os pontos mais próximos, encontra-se a equação da reta que intersecta esses dois pontos com complexidade de tempo <i>O</i>(1). Obtendo essa equação da reta, encontra o ponto médio dessa reta com complexidade de tempo <i>O</i>(1). Após isso, encontra a equação da reta que é perpendicular a essa equação da reta, passando pelo ponto médio, com complexidade de tempo <i>O</i>(1).
<p>Com o valor da equação da reta perpendicular, agora é utilizar o classificador determinar de qual classe pontos desconhecidos pertencem. Para isso, o classificador testa se o ponto é maior que 0 ou menor que 0 para determinar de qual classe ele faz parte. Sua complexidade de tempo é <i>O</i>(p), sendo p o número de pontos a serem classificados.

<h1>Experimentos</h1>

Foi feito 10 experimentos, com os dados: <i>australian, bands, bupa, haberman, heart, hepatitis, mammographic, pima, saheart</i> e <i>titanic</i>.
<p>Dessas bases de dados, 6 foram linearmente separáveis, 2 dessas a equação perpendicular intersecta alguma das envoltórias e 2 dessas as envoltórias se intersectam. obs: os gráficos estão um pouco distorcidos, mas a reta é perpendicular ao ponto médio em todos os casos.
<p>A seguir, estão os gráficos de cada base de dados:

<h4>australian</h4>

<img src="australian/australian.png"> 

<h4>bands</h4>

<img src="bands/bands.png">

<h4>bupa</h4>

<img src="bupa/bupa.png">

<h4>haberman</h4>

<img src="haberman/haberman.png">

<h4>heart</h4>

<img src="heart/heart.png">

<h4>hepatitis</h4>

<img src="hepatitis/hepatitis.png">

<h4>mammographic</h4>

<img src="mammographic/mammographic.png">

<h4>pima</h4>

<img src="pima/pima.png">

<h4>saheart</h4>


<img src="saheart/saheart.png">

<h4>titanic</h4>

<img src="titanic/titanic.png">

<h2>Precisão, Revocação e F1-score</h2>

Utilizando a biblioteca <i>Scikit-learn</i>, foi usado a função <i>train_test_split</i> para treinar 70% dos dados e prever os outros 30% desconhecidos.<br>
Após isso, foi medido a precisão, revocação e f1-score, obtendo:

base de dados|Precisão|Revocação|F1-score
:--------:|:--------:|:---------:|:--------:
australian| | | 
bands| | | 
bupa|0.582524|0.582524|0.582524 
haberman| | | 
heart|0.436893|0.436893|0.436893
hepatitis|0.823529|0.823529|0.823529
mammographic|0.491428|0.491428|0.491428
pima| | | 
saheart|0.287179 | 0.287179| 0.287179
titanic|0.677761|0.677761|0.677761